# How to Train Your Own Cone Detection Networks

![](https://user-images.githubusercontent.com/22118253/70957091-fe06a480-2042-11ea-8c06-0fcc549fc19a.png)

In this notebook, we will demonstrate 
- how to train your own YOLOv3-based traffic cone detection network and do inference on a video.

**[Accurate Low Latency Visual Perception for Autonomous Racing: Challenges Mechanisms and Practical Solutions](https://github.com/mit-han-lab/once-for-all)** is an accurate low latency visual perception system introduced by Kieran Strobel, Sibo Zhu, Raphael Chang, and Skanda Koppula.

## 1. Preparation
Let's first install all the required packages:

In [ ]:
! sudo apt install unzip
print('Installing PyTorch...')
! pip3 install torch 
print('Installing torchvision...')
! pip3 install torchvision 
print('Installing numpy...')
! pip3 install numpy 
# tqdm is a package for displaying a progress bar.
print('Installing tqdm (progress bar) ...')
! pip3 install tqdm 
print('Installing matplotlib...')
! pip3 install matplotlib 
print('Installing all the other required packages once for all')
! sudo python3 setup.py install
print('Installing video editor')
! sudo apt install ffmpeg -y 

Before we start training, let's download the Cone Detection dataset and the corresponding label and intial training weights. 

In [ ]:
print("Downloading Training Dataset")
! wget https://storage.googleapis.com/mit-driverless-open-source/YOLO_Dataset.zip
! unzip YOLO_Dataset.zip
! mv YOLO_Dataset dataset/ && rm YOLO_Dataset.zip
print("Downloading YOLOv3 Sample Weights")
! wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/sample-yolov3.weights 
print("Downloading Training and Validation Label")
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/all.csv && cd ..
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/train.csv && cd ..
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/validate.csv && cd ..

## 2. Using Pretrained YOLOv3 Weights File to Start Training


First, import all the packages used in this tutorial:

In [6]:
import os
import random
import tempfile
import time
import multiprocessing
import subprocess
import math
import shutil
import math

from datetime import datetime
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from models import Darknet
from utils.datasets import ImageLabelDataset
from utils.utils import model_info, print_args, Logger, visualize_and_save_to_local,xywh2xyxy
import validate
import warnings
import sys

##### section for all random seeds #####
torch.manual_seed(17)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
########################################

warnings.filterwarnings("ignore")
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')
num_cpu = multiprocessing.cpu_count() if cuda else 0
if cuda:
    torch.cuda.synchronize()
random.seed(0)
torch.manual_seed(0)

if cuda:
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    torch.backends.cudnn.benchmark = True
    torch.cuda.empty_cache()

Successfully imported all packages and configured random seed to 17!

In [7]:
def run_epoch(label_prefix, data_loader, num_steps, optimizer, model, epoch, num_epochs, step):
    print(f"Model in {label_prefix} mode")
    epoch_losses = [0.0] * 7
    epoch_time_total = 0.0
    epoch_num_targets = 1e-12
    t1 = time.time()
    loss_labels = ["Total", "L-x", "L-y", "L-w", "L-h", "L-noobj", "L-obj"]
    for i, (img_uri, imgs, targets) in enumerate(data_loader):
        if step[0] >= num_steps:
            break
        imgs = imgs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        targets.requires_grad_(False)
        step_num_targets = ((targets[:, :, 1:5] > 0).sum(dim=2) > 1).sum().item() + 1e-12
        epoch_num_targets += step_num_targets
        # Compute loss, compute gradient, update parameters
        if optimizer is not None:
            optimizer.zero_grad()
        losses = model(imgs, targets)
        if label_prefix == "train":
            losses[0].sum().backward()
        if optimizer is not None:
            optimizer.step()

        for j, (label, loss) in enumerate(zip(loss_labels, losses)):
            batch_loss = loss.sum().to('cpu').item()
            epoch_losses[j] += batch_loss
        finished_time = time.time()
        step_time_total = finished_time - t1
        epoch_time_total += step_time_total
        
        statement = label_prefix + ' Epoch: ' + str(epoch) + ', Batch: ' + str(i + 1) + '/' + str(len(data_loader))
        count = 0
        for (loss_label, loss) in zip(loss_labels, losses):
            if count == 0:
                statement += ', Total: ' + '{0:10.6f}'.format(loss.item() / step_num_targets)
                tot_loss = loss.item()
                count += 1
            else:
                statement += ',   ' + loss_label + ': {0:5.2f}'.format(loss.item() / tot_loss * 100) + '%'
        print(statement)
        if label_prefix == "train":
            step[0] += 1
    return epoch_losses, epoch_time_total, epoch_num_targets

Training Config

In [8]:
evaluate = False
batch_size = int(5)
optimizer_pick = "Adam"
model_cfg = "model_cfg/yolo_baseline.cfg"
weights_path = "sample-yolov3.weights"
output_path = "automatic"
dataset_path = "dataset/YOLO_Dataset/"
num_epochs = int(2048)
num_steps = 8388608
checkpoint_interval = int(1)
augment_affine = False
augment_hsv = False
lr_flip = False
ud_flip = False
momentum = float(0.9)
gamma = float(0.95)
lr = float(0.001)
weight_decay = float(0.0)
vis_batch = int(0)
data_aug = False
blur = False
salt = False
noise = False
contrast = False
sharpen = False
ts = True
debug_mode = False
upload_dataset = False
xy_loss = float(2)
wh_loss= float(1.6)
no_object_loss = float(25)
object_loss = float(0.1)
vanilla_anchor = False
val_tolerance = int(3)
min_epochs = int(3)

In [ ]:
input_arguments = list(locals().items())

print("Initializing model")
model = Darknet(config_path=model_cfg,xy_loss=xy_loss,wh_loss=wh_loss,no_object_loss=no_object_loss,object_loss=object_loss,vanilla_anchor=vanilla_anchor)
img_width, img_height = model.img_size()
bw  = model.get_bw()
validate_uri, train_uri = model.get_links()

if output_path == "automatic":
    current_month = datetime.now().strftime('%B').lower()
    current_year = str(datetime.now().year)
    if not os.path.exists(os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1])):
        os.makedirs(os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1]))
    output_uri = os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1])
else:
    output_uri = output_path

num_validate_images, num_train_images = model.num_images()
conf_thresh, nms_thresh, iou_thresh = model.get_threshs()
num_classes = model.get_num_classes()
loss_constant = model.get_loss_constant()
conv_activation = model.get_conv_activation()
anchors = model.get_anchors()
onnx_name = model.get_onnx_name()

with tempfile.TemporaryDirectory() as tensorboard_data_dir:
    print("Initializing data loaders")
    train_data_loader = torch.utils.data.DataLoader(
        ImageLabelDataset(train_uri, dataset_path=dataset_path, width=img_width, height=img_height, augment_hsv=augment_hsv,
                            augment_affine=augment_affine, num_images=num_train_images,
                            bw=bw, n_cpu=num_cpu, lr_flip=lr_flip, ud_flip=ud_flip,vis_batch=vis_batch,data_aug=data_aug,blur=blur,salt=salt,noise=noise,contrast=contrast,sharpen=sharpen,ts=ts,debug_mode=debug_mode, upload_dataset=upload_dataset),
        batch_size=(1 if debug_mode else batch_size),
        shuffle=(False if debug_mode else True),
        num_workers=(0 if vis_batch else num_cpu),
        pin_memory=cuda)
    print("Num train images: ", len(train_data_loader.dataset))

    validate_data_loader = torch.utils.data.DataLoader(
        ImageLabelDataset(validate_uri, dataset_path=dataset_path, width=img_width, height=img_height, augment_hsv=False,
                            augment_affine=False, num_images=num_validate_images,
                            bw=bw, n_cpu=num_cpu, lr_flip=False, ud_flip=False,vis_batch=vis_batch,data_aug=False,blur=False,salt=False,noise=False,contrast=False,sharpen=False,ts=ts,debug_mode=debug_mode, upload_dataset=upload_dataset),
        batch_size=(1 if debug_mode else batch_size),
        shuffle=False,
        num_workers=(0 if vis_batch else num_cpu),
        pin_memory=cuda)
    print("Num validate images: ", len(validate_data_loader.dataset))

    ##### additional configuration #####
    print("Training batch size: " + str(batch_size))
    
    print("Checkpoint interval: " + str(checkpoint_interval))

    print("Loss constants: " + str(loss_constant))

    print("Anchor boxes: " + str(anchors))

    print("Training image width: " + str(img_width))

    print("Training image height: " + str(img_height))

    print("Confidence Threshold: " + str(conf_thresh))

    print("Number of training classes: " + str(num_classes))

    print("Conv activation type: " + str(conv_activation))

    print("Starting learning rate: " + str(lr))

    if ts:
        print("Tile and scale mode [on]")
    else:
        print("Tile and scale mode [off]")

    if data_aug:
        print("Data augmentation mode [on]")
    else:
        print("Data augmentation mode [off]")

    ####################################

    start_epoch = 0

    weights_path = weights_path
    if optimizer_pick == "Adam":
        print("Using Adam Optimizer")
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                    lr=lr, weight_decay=weight_decay)
    elif optimizer_pick == "SGD":
        print("Using SGD Optimizer")
        optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                                lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise Exception(f"Invalid optimizer name: {optimizer_pick}")
    print("Loading weights")
    model.load_weights(weights_path, model.get_start_weight_dim())

    if torch.cuda.device_count() > 1:
        print('Using ', torch.cuda.device_count(), ' GPUs')
        model = nn.DataParallel(model)
    model = model.to(device, non_blocking=True)

    # Set scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)

    val_loss = 999  # using a high number for validation loss
    val_loss_counter = 0
    step = [0]  # wrapping in an array so it is mutable
    epoch = start_epoch
    while epoch < num_epochs and step[0] < num_steps and not evaluate:
        epoch += 1
        scheduler.step()
        model.train()
        run_epoch(label_prefix="train", data_loader=train_data_loader, epoch=epoch,
                    step=step, model=model, num_epochs=num_epochs, num_steps=num_steps,
                    optimizer=optimizer)
        print('Completed epoch: ', epoch)
        # Update best loss
        if epoch % checkpoint_interval == 0 or epoch == num_epochs or step[0] >= num_steps:
            # First, save the weights
            save_weights_uri = os.path.join(output_uri, "{epoch}.weights".format(epoch=epoch))
            model.save_weights(save_weights_uri)

            with torch.no_grad():
                print("Calculating loss on validate data")
                epoch_losses, epoch_time_total, epoch_num_targets = run_epoch(
                    label_prefix="validate", data_loader=validate_data_loader, epoch=epoch,
                    model=model, num_epochs=num_epochs, num_steps=num_steps, optimizer=None,
                    step=step)
                avg_epoch_loss = epoch_losses[0] / epoch_num_targets
                print('Average Validation Loss: {0:10.6f}'.format(avg_epoch_loss))

                if avg_epoch_loss > val_loss and epoch > min_epochs:
                    val_loss_counter += 1
                    print(f"Validation loss did not decrease for {val_loss_counter}"
                            f" consecutive check(s)")
                else:
                    print("Validation loss decreased. Yay!!")
                    val_loss_counter = 0
                    val_loss = avg_epoch_loss
                    ##### updating best result for optuna study #####
                    result = open("logs/result.txt", "w" )
                    result.write(str(avg_epoch_loss))
                    result.close() 
                    ###########################################
                validate.validate(dataloader=validate_data_loader, model=model, device=device, step=step[0], bbox_all=False,debug_mode=debug_mode)
                if val_loss_counter == val_tolerance:
                    print("Validation loss stopped decreasing over the last " + str(val_tolerance) + " checkpoints, creating onnx file")
                    with tempfile.NamedTemporaryFile() as tmpfile:
                        model.save_weights(tmpfile.name)
                        weights_name = tmpfile.name
                        cfg_name = os.path.join(tempfile.gettempdir(), model_cfg.split('/')[-1].split('.')[0] + '.tmp')
                        onnx_gen = subprocess.call(['python3', 'yolo2onnx.py', '--cfg_name', cfg_name, '--weights_name', weights_name])
                        save_weights_uri = os.path.join(output_uri, onnx_name)
                        os.rename(weights_name, save_weights_uri)
                        try:
                            os.remove(onnx_name)
                        except:
                            pass
                        os.remove(cfg_name)
                    break
    if evaluate:
        validation = validate.validate(dataloader=validate_data_loader, model=model, device=device, step=-1, bbox_all=False, tensorboard_writer=None,debug_mode=debug_mode)
return val_loss


## 3. Inference

Download target video file for inference

In [10]:
from IPython.display import Video

! wget https://storage.googleapis.com/mit-driverless-open-source/test_video.mp4

! ffmpeg -i test_video.mp4 test.mp4 && rm test_video.mp4

Video("test.mp4")

--2020-08-25 13:54:22--  https://storage.googleapis.com/mit-driverless-open-source/test_video.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 172.253.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12062655 (12M) [video/mp4]
Saving to: ‘test_video.mp4’

test_video.mp4      100%[===================>]  11.50M  --.-KB/s    in 0.04s   

2020-08-25 13:54:22 (273 MB/s) - ‘test_video.mp4’ saved [12062655/12062655]

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --

Download pretrained weights for inference

In [11]:
! wget https://storage.googleapis.com/mit-driverless-open-source/pretrained_yolo.weights

--2020-08-25 13:54:30--  https://storage.googleapis.com/mit-driverless-open-source/pretrained_yolo.weights
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 172.253.119.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘pretrained_yolo.weights’

pretrained_yolo.wei 100%[===================>] 236.52M   105MB/s    in 2.3s    

2020-08-25 13:54:33 (105 MB/s) - ‘pretrained_yolo.weights’ saved [248007048/248007048]



Import all packages for inference

In [12]:
import os
from os.path import isfile, join
import copy
import cv2
from tensorboardX import SummaryWriter
from PIL import Image, ImageDraw
import torchvision
from utils.nms import nms
from utils.utils import calculate_padding
from tqdm import tqdm

In [13]:
warnings.filterwarnings("ignore")
detection_tmp_path = "/tmp/detect/"

Set up config file for inference

In [14]:
target_path = "test.mp4"
output_path = "outputs/visualization/"
weights_path = "pretrained_yolo.weights"
conf_thres = float(0.8)
nms_thres = float(0.25)

In [15]:
def single_img_detect(target_path,output_path,mode,model,device,conf_thres,nms_thres):

    img = Image.open(target_path).convert('RGB')
    w, h = img.size
    new_width, new_height = model.img_size()
    pad_h, pad_w, ratio = calculate_padding(h, w, new_height, new_width)
    img = torchvision.transforms.functional.pad(img, padding=(pad_w, pad_h, pad_w, pad_h), fill=(127, 127, 127), padding_mode="constant")
    img = torchvision.transforms.functional.resize(img, (new_height, new_width))

    bw = model.get_bw()
    if bw:
        img = torchvision.transforms.functional.to_grayscale(img, num_output_channels=1)

    img = torchvision.transforms.functional.to_tensor(img)
    img = img.unsqueeze(0)
    
    with torch.no_grad():
        model.eval()
        img = img.to(device, non_blocking=True)
        # output,first_layer,second_layer,third_layer = model(img)
        output = model(img)


        for detections in output:
            detections = detections[detections[:, 4] > conf_thres]
            box_corner = torch.zeros((detections.shape[0], 4), device=detections.device)
            xy = detections[:, 0:2]
            wh = detections[:, 2:4] / 2
            box_corner[:, 0:2] = xy - wh
            box_corner[:, 2:4] = xy + wh
            probabilities = detections[:, 4]
            nms_indices = nms(box_corner, probabilities, nms_thres)
            main_box_corner = box_corner[nms_indices]
            if nms_indices.shape[0] == 0:  
                continue
        img_with_boxes = Image.open(target_path)
        draw = ImageDraw.Draw(img_with_boxes)
        w, h = img_with_boxes.size

        for i in range(len(main_box_corner)):
            x0 = main_box_corner[i, 0].to('cpu').item() / ratio - pad_w
            y0 = main_box_corner[i, 1].to('cpu').item() / ratio - pad_h
            x1 = main_box_corner[i, 2].to('cpu').item() / ratio - pad_w
            y1 = main_box_corner[i, 3].to('cpu').item() / ratio - pad_h 
            draw.rectangle((x0, y0, x1, y1), outline="red")

        if mode == 'image':
            img_with_boxes.save(os.path.join(output_path,target_path.split('/')[-1]))
            return os.path.join(output_path,target_path.split('/')[-1])
        else:
            img_with_boxes.save(target_path)
            return target_path

In [16]:
def detect(target_path,
           output_path,
           model,
           device,
           conf_thres,
           nms_thres):

        target_filepath = target_path

        img_formats = ['.jpg', '.jpeg', '.png', '.tif']
        vid_formats = ['.mov', '.avi', '.mp4']

        mode = None

        if os.path.splitext(target_filepath)[-1].lower() in img_formats:
            mode = 'image'
        
        elif os.path.splitext(target_filepath)[-1].lower() in vid_formats:
            mode = 'video'
        
        print("Detection Mode is: " + mode)

        raw_file_name = target_filepath.split('/')[-1].split('.')[0].split('_')[-4:]
        raw_file_name = '_'.join(raw_file_name)
        
        if mode == 'image':
            detection_path = single_img_detect(target_path=target_filepath,output_path=output_path,mode=mode,model=model,device=device,conf_thres=conf_thres,nms_thres=nms_thres)

            print(f'Please check output image at {detection_path}')

        elif mode == 'video':
            if os.path.exists(detection_tmp_path):
                shutil.rmtree(detection_tmp_path)  # delete output folder
            os.makedirs(detection_tmp_path)  # make new output folder

            vidcap = cv2.VideoCapture(target_filepath)
            success,image = vidcap.read()
            count = 0

            

            while success:
                cv2.imwrite(detection_tmp_path + "/frame%d.jpg" % count, image)     # save frame as JPEG file      
                success,image = vidcap.read()
                count += 1

            # Find OpenCV version
            (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

            if int(major_ver)  < 3 :
                fps = vidcap.get(cv2.cv.CV_CAP_PROP_FPS)
                print ("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
            else :
                fps = vidcap.get(cv2.CAP_PROP_FPS)
                print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
            vidcap.release(); 

            frame_array = []
            files = [f for f in os.listdir(detection_tmp_path) if isfile(join(detection_tmp_path, f))]
        
            #for sorting the file names properly
            files.sort(key = lambda x: int(x[5:-4]))
            for i in tqdm(files,desc='Doing Single Image Detection'):
                filename=detection_tmp_path + i
                
                detection_path = single_img_detect(target_path=filename,output_path=output_path,mode=mode,model=model,device=device,conf_thres=conf_thres,nms_thres=nms_thres)
                #reading each files
                img = cv2.imread(detection_path)
                height, width, layers = img.shape
                size = (width,height)
                frame_array.append(img)

            local_output_uri = output_path + raw_file_name + ".mp4"
            
            video_output = cv2.VideoWriter(local_output_uri,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

            for frame in tqdm(frame_array,desc='Creating Video'):
                # writing to a image array
                video_output.write(frame)
            video_output.release()
            shutil.rmtree(detection_tmp_path)

In [17]:
cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')
random.seed(0)
torch.manual_seed(0)
if cuda:
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    torch.backends.cudnn.benchmark = True
    torch.cuda.empty_cache()
model = Darknet(config_path=model_cfg,xy_loss=xy_loss,wh_loss=wh_loss,no_object_loss=no_object_loss,object_loss=object_loss,vanilla_anchor=vanilla_anchor)

# Load weights
model.load_weights(weights_path, model.get_start_weight_dim())
model.to(device, non_blocking=True)

detect(target_path,
        output_path,
        model,
        device=device,
        conf_thres=conf_thres,
        nms_thres=nms_thres)

Detection Mode is: video


Doing Single Image Detection:   0%|          | 0/646 [00:00<?, ?it/s]

Frames per second using video.get(cv2.CAP_PROP_FPS) : 26.0


Creating Video: 100%|██████████| 646/646 [00:03<00:00, 184.07it/s]


In [18]:
! cd outputs/visualization/ && ffmpeg -i test.mp4 output.mp4 && rm test.mp4 && cd ../..

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [19]:
Video("outputs/visualization/output.mp4")

**Notice:** Again, you can further improve the accuracy of the cone detection network by switching YOLOv3 backbone to the most recent published YOLOv4

![](https://user-images.githubusercontent.com/22118253/70950893-e2de6980-202f-11ea-9a16-399579926ee5.gif)

Congratulations! You've finished all the content of this tutorial!
Hope you enjoy playing with the our object detection model. If you are interested,  please refer to our paper and GitHub Repo for further details.

## Reference
[1] Kieran Strobel, Sibo Zhu, Raphael Chang and Skanda Koppula.
**Accurate, Low-Latency Visual Perception for Autonomous Racing:Challenges, Mechanisms, and Practical Solutions**. In *IROS* 2020.
[[paper]](https://arxiv.org/abs/2007.13971), [[code]](https://github.com/cv-core/MIT-Driverless-CV-TrainingInfra).